## Run dependant pipelines

In [16]:
!{'cd /pipelines; KNESSET_LOAD_FROM_URL=1 dpp run --verbose ./people/committee-meeting-attendees'}

[./people/committee-meeting-attendees:T_0] >>> INFO    :1701d501 RUNNING ./people/committee-meeting-attendees
[./people/committee-meeting-attendees:T_0] >>> INFO    :1701d501 Collecting dependencies
[./people/committee-meeting-attendees:T_0] >>> INFO    :1701d501 Running async task
[./people/committee-meeting-attendees:T_0] >>> INFO    :1701d501 Waiting for completion
[./people/committee-meeting-attendees:T_0] >>> INFO    :1701d501 Async task starting
[./people/committee-meeting-attendees:T_0] >>> INFO    :1701d501 Searching for existing caches
[./people/committee-meeting-attendees:T_0] >>> INFO    :1701d501 Building process chain:
[./people/committee-meeting-attendees:T_0] >>> INFO    :- load_resource
[./people/committee-meeting-attendees:T_0] >>> INFO    :- dump.to_path
[./people/committee-meeting-attendees:T_0] >>> INFO    :- (sink)
[./people/committee-meeting-attendees:T_0] >>> INFO    :load_resource: DEBUG   :Starting new HTTP connection (1): storage.googleapis.com:80
[./people/co

## Add a filter step

You should uncomment / add a filter step to committee-meeting-speaker-stats pipeline:

```
  - run: filter
    cache: true
    parameters:
      resources: kns_committeesession
      in:
      - CommitteeSessionID: 2059313
  - run: committee_meeting_speaker_stats
```

## Download protocol parts files

upgrade to latest dataflows library

In [ ]:
!{'pip install --upgrade dataflows'}

Restart the kernel if an upgrade was done

Choose some session IDs to download protocol files for:

In [5]:
session_ids = [2059313,86485,2058899,2081445]

In [6]:
from dataflows import Flow, load, printer, filter_rows

sessions_data = Flow(
    load('/pipelines/data/committees/kns_committeesession/datapackage.json'),
    filter_rows(lambda row: row['CommitteeSessionID'] in session_ids),
    printer(tablefmt='html')
).results()

#,CommitteeSessionID (integer),Number (integer),KnessetNum (integer),TypeID (integer),TypeDesc (string),CommitteeID (integer),Location (string),SessionUrl (string),BroadcastUrl (string),StartDate (datetime),FinishDate (datetime),Note (string),LastUpdatedDate (datetime),download_crc32c (string),download_filename (string),download_filesize (integer),parts_crc32c (string),parts_filesize (integer),parts_parsed_filename (string),text_crc32c (string),text_filesize (integer),text_parsed_filename (string),topics (array),committee_name (string)
1,86485,334,16,161,פתוחה,23,"חדר הוועדה, באגף הוועדות (קדמה), קומה 3, חדר 3740",http://main.knesset.gov.il/Activity/committees/Pages/AllCommitteesAgenda.aspx?Tab=3&ItemID=86485,None,2004-12-27 11:30:00,None,None,2012-09-19 15:27:34,8zHRDQ==,files/23/7/1/71955.DOC,38912,UloMhw==,9385,files/8/6/86485.csv,Inn+AQ==,9342,files/8/6/86485.txt,"['חוק רשות התעופה האזרחית, התשס""ה-2005']",הכלכלה
2,2058899,51,15,161,פתוחה,2,"חדר הוועדה, באגף הוועדות (קדמה), קומה 3, חדר 3750",http://main.knesset.gov.il/Activity/committees/Pages/AllCommitteesAgenda.aspx?Tab=3&ItemID=2058899,None,1999-11-25 00:00:00,1999-11-25 12:15:00,תקציב המשרד לקליטת העלייה לשנת 2002.,2018-10-10 11:03:06,L9/m4w==,files/23/4/5/450720.DOC,51456,yfFdlw==,71227,files/2/0/2058899.csv,UjcTNA==,71956,files/2/0/2058899.txt,None,הכספים
3,2059313,462,15,161,פתוחה,2,"חדר הוועדה, באגף הוועדות (קדמה), קומה 3, חדר 3750",http://main.knesset.gov.il/Activity/committees/Pages/AllCommitteesAgenda.aspx?Tab=3&ItemID=2059313,None,2002-11-19 00:00:00,2002-11-19 00:00:00,"הצעת חוק ההסדרים במשק המדינה (תיקוני חקיקה להשגת יעדי התקציב והמדיניות הכלכלית לשנת הכספים 2003, התש ...",2018-10-10 11:03:06,+lu4+A==,files/23/4/3/430592.DOC,91162,x9rEiQ==,194576,files/2/0/2059313.csv,jD1Riw==,195031,files/2/0/2059313.txt,None,הכספים
4,2081445,1,22,161,פתוחה,2109,None,http://main.knesset.gov.il/Activity/committees/Pages/AllCommitteesAgenda.aspx?Tab=3&ItemID=2081445,http://main.knesset.gov.il/Activity/committees/Pages/AllCommitteesBroadcast.aspx?TopicID=19806,2019-10-03 17:30:00,None,None,2019-10-03 22:48:36,None,None,None,None,None,None,None,None,None,"['בחירת ועדה זמנית לענייני כספים ולענייני חוץ וביטחון והמלצה על יושב ראש הוועדות', 'בחירת סגנים זמני ...",הוועדה המסדרת


In [8]:
import os
import subprocess
import sys

for session in sessions_data[0][0]:
    for attr in ['text_parsed_filename', 'parts_parsed_filename']:
        filename = session[attr]
        if filename:
            pathpart = 'meeting_protocols_text' if attr == 'text_parsed_filename' else 'meeting_protocols_parts'
            url = 'https://production.oknesset.org/pipelines/data/committees/{}/{}'.format(pathpart, filename)
            filename = '/pipelines/data/committees/{}/{}'.format(pathpart, session[attr])
            os.makedirs(os.path.dirname(filename), exist_ok=True)
            cmd = 'curl -s -o {} {}'.format(filename, url)
            print(cmd, file=sys.stderr)
            subprocess.check_call(cmd, shell=True)

curl -s -o /pipelines/data/committees/meeting_protocols_text/files/8/6/86485.txt https://production.oknesset.org/pipelines/data/committees/meeting_protocols_text/files/8/6/86485.txt
curl -s -o /pipelines/data/committees/meeting_protocols_parts/files/8/6/86485.csv https://production.oknesset.org/pipelines/data/committees/meeting_protocols_parts/files/8/6/86485.csv
curl -s -o /pipelines/data/committees/meeting_protocols_text/files/2/0/2058899.txt https://production.oknesset.org/pipelines/data/committees/meeting_protocols_text/files/2/0/2058899.txt
curl -s -o /pipelines/data/committees/meeting_protocols_parts/files/2/0/2058899.csv https://production.oknesset.org/pipelines/data/committees/meeting_protocols_parts/files/2/0/2058899.csv
curl -s -o /pipelines/data/committees/meeting_protocols_text/files/2/0/2059313.txt https://production.oknesset.org/pipelines/data/committees/meeting_protocols_text/files/2/0/2059313.txt
curl -s -o /pipelines/data/committees/meeting_protocols_parts/files/2/0/20

## Run the pipeline

In [ ]:
!{'rm -rf /pipelines/data/people/committees/meeting-speaker-stats/cache_hash'}

In [26]:
!{'cd /pipelines; dpp run --verbose ./people/committee-meeting-speaker-stats'}

[./people/committee-meeting-speaker-stats:T_0] >>> INFO    :76ef1179 RUNNING ./people/committee-meeting-speaker-stats
[./people/committee-meeting-speaker-stats:T_0] >>> INFO    :76ef1179 Collecting dependencies
[./people/committee-meeting-speaker-stats:T_0] >>> INFO    :76ef1179 Running async task
[./people/committee-meeting-speaker-stats:T_0] >>> INFO    :76ef1179 Waiting for completion
[./people/committee-meeting-speaker-stats:T_0] >>> INFO    :76ef1179 Async task starting
[./people/committee-meeting-speaker-stats:T_0] >>> INFO    :76ef1179 Searching for existing caches
[./people/committee-meeting-speaker-stats:T_0] >>> INFO    :Found cache for step 1: filter
[./people/committee-meeting-speaker-stats:T_0] >>> INFO    :76ef1179 Building process chain:
[./people/committee-meeting-speaker-stats:T_0] >>> INFO    :- cache_loader
[./people/committee-meeting-speaker-stats:T_0] >>> INFO    :- committee_meeting_speaker_stats
[./people/committee-meeting-speaker-stats:T_0] >>> INFO    :- knesse

## Inspect the output

In [27]:
from dataflows import Flow, load, printer
from collections import defaultdict

session_parts_stats = {}
header_parts_stats = {}

def aggregate_speaker_session_stats(rows):
    if rows.res.name == 'speaker_stats':
        for row in rows:
            session = sessions_data
            session_stats_row = session_parts_stats.setdefault(row['CommitteeSessionID'], defaultdict(int))
            session_stats_row['num_parts'] += 1
            headers_stats_row = header_parts_stats.setdefault(row['header'], defaultdict(int))
            headers_stats_row['num_parts'] += 1
    yield from rows

Flow(
    load('/pipelines/data/people/committees/meeting-speaker-stats/datapackage.json'),
    aggregate_speaker_session_stats,
).process()

(<datapackage.package.Package at 0x7ff00e1460b8>, {})

## Session speaker stats

In [29]:
Flow(
    ({
        **{k: session[k] for k in ['CommitteeSessionID', 'KnessetNum', 'StartDate', 'committee_name', 'topics', 'Note']},
        **session_parts_stats[session['CommitteeSessionID']],
        'oknesset_url': 'https://oknesset.org/meetings/{}/{}/{}.html'.format(str(session['CommitteeSessionID'])[0],
                                                                             str(session['CommitteeSessionID'])[1],
                                                                             str(session['CommitteeSessionID']))
    } for session in sessions_data[0][0] if session['CommitteeSessionID'] in session_parts_stats),
    printer(tablefmt='html')
).process()

#,CommitteeSessionID (integer),KnessetNum (integer),StartDate (datetime),committee_name (string),topics (array),Note (string),num_parts (integer),oknesset_url (string)
1,86485,16,2004-12-27 11:30:00,הכלכלה,"['חוק רשות התעופה האזרחית, התשס""ה-2005']",None,29,https://oknesset.org/meetings/8/6/86485.html
2,2058899,15,1999-11-25 00:00:00,הכספים,None,תקציב המשרד לקליטת העלייה לשנת 2002.,128,https://oknesset.org/meetings/2/0/2058899.html
3,2059313,15,2002-11-19 00:00:00,הכספים,None,"הצעת חוק ההסדרים במשק המדינה (תיקוני חקיקה להשגת יעדי התקציב והמדיניות הכלכלית לשנת הכספים 2003, התש ...",435,https://oknesset.org/meetings/2/0/2059313.html


(<datapackage.package.Package at 0x7ff00de4ef98>, {})

## Header stats

In [31]:
Flow(
    ({
        'header': header,
        **headers_stats_row
    } for header, headers_stats_row in header_parts_stats.items()
    if header and 'יעל תמיר' in header),
    printer(tablefmt='html')
).process()

#,header (string),num_parts (integer)
1,שרת הקליטה יעל תמיר,25
2,שרת העליה והקליטה יעל תמיר,1
3,שר הקליטה והעליה יעל תמיר,1


(<datapackage.package.Package at 0x7ff00de97160>, {})